In [41]:
import requests
from  bs4 import BeautifulSoup
import re
from datetime import datetime
import pandas as pd

def getListingListFromSoup()->list:
    def getSoup()->BeautifulSoup:
        html_text = requests.get('https://www.kucoin.com/fr/news/categories/listing',headers={'Accept':'*/*','User-Agent':'Thunder Client (https://www.thunderclient.com)'}).text
        return BeautifulSoup(html_text, 'html.parser')
    return [listing.text.strip() for listing in getSoup().find_all('div',{'class':'mainTitle___mbpq1'})]

Listing = getListingListFromSoup()

     
def cleanUpAndFormatListing(listings):
    def keepOnlyCurrencies(_listings:list)->list:
        return [listing for listing in _listings if 'etf' not in listing.lower()] 
    
    listings = keepOnlyCurrencies(listings)
    
    def extractSymbols(_listings):
        return [ re.search('(?<=\()[a-zA-Z]+?(?=\))',
                           re.sub(r'Trading:\s(.*)$',
                                  '', listing)).group(0)
                for listing in _listings]
    
    def extractDates(_listings):
        return [ datetime.strptime(re.search(r'Trading:\s(.*)$',
                                                 listing).group(1),
                                       '%H:%M on %B %d, %Y (UTC)')
                for listing in _listings]
        
    dates = extractDates(listings)    
    symbols = extractSymbols(listings)
    return pd.DataFrame({'symbol':symbols,'date':dates})
    
df_listing = cleanUpAndFormatListing(Listing)

df_listing.loc[df_listing['date']>datetime.now()]   

,symbol,date
0,NEER,2022-08-30 12:00:00
